In [101]:
import pandas as pd 
import numpy as np
import copy

#data = pd.read_csv("train.csv") 
#ran = np.random.rand(len(data)) < 0.8
#train = data[ran]
#val = data[~ran]
#train.to_csv('UsedTrainingData.csv')
#val.to_csv('UsedValidationData.csv')

train =  pd.read_csv('UsedTrainingData.csv')
val = pd.read_csv('UsedValidationData.csv')

train = train.loc[:,['Work_accident', 'promotion_last_5years', 'sales', 'salary','left'] ] 
val = val.loc[:,['Work_accident', 'promotion_last_5years', 'sales', 'salary','left'] ] 

#train = train.drop_duplicates(subset=None, keep=False, inplace=False)
#val = val.drop_duplicates(subset=None, keep=False, inplace=False)

targetAttribute = 'left'
Attributes = list(train.columns)
Attributes.remove(targetAttribute)
examples = list(train.index.values) 

NumericalAttributes = ['satisfaction_level', 'last_evaluation', 'number_project', 'average_montly_hours', 'time_spend_company']
CategoricalAttributes = ['Work_accident', 'promotion_last_5years', 'sales', 'salary']

def getImpurityOfExamples(examplesForCurNode,method):
    NumberOfPosEx = 0
    NumberOfNegEx = 0
    for j in examplesForCurNode:
        if  str(train.at[int(j),targetAttribute]) == str(1):
            NumberOfPosEx = NumberOfPosEx+1
        elif str(train.at[int(j),targetAttribute]) == str(0):
             NumberOfNegEx = NumberOfNegEx+1
    PropOfPos =  NumberOfPosEx/len(examplesForCurNode)
    PropOfNeg = NumberOfNegEx/len(examplesForCurNode)
    if method == 'Gini':
        return (2*PropOfPos*PropOfNeg)
    elif method == 'misClasifi':
        return min(PropOfPos,PropOfNeg)
    elif method == 'Entropy':
        if PropOfPos == 0:
            return (-1*PropOfNeg*(np.log(PropOfNeg)/np.log(2)))
        elif PropOfNeg == 0:
            return (-1*PropOfPos*(np.log(PropOfPos)/np.log(2)))
        else:
            return ((-1*PropOfPos*(np.log(PropOfPos)/np.log(2))) + (-1*PropOfNeg*(np.log(PropOfNeg)/np.log(2))))

def getMethodImpurity(Attribute,value,examplesForCurAttr,method): 
    fra1 = train.loc[examplesForCurAttr,:]
    LessThanExamples = []
    MoreThanExamples = []
    if Attribute in NumericalAttributes: 
        for j in examplesForCurAttr:
            if  float(str(fra1.at[int(j),Attribute])) <= float(str(value)):
                LessThanExamples.append(int(j))
            else:
                MoreThanExamples.append(int(j))
    else:
        for j in examplesForCurAttr:
            if  str(fra1.at[int(j),Attribute]) == str(value):
                LessThanExamples.append(int(j))
            else:
                MoreThanExamples.append(int(j))
        if len(MoreThanExamples) <= 0:
            return 999
        elif len(LessThanExamples) <=0:
            return getImpurityOfExamples(MoreThanExamples,method)
        else:            
            return ((len(LessThanExamples)*getImpurityOfExamples(LessThanExamples,method))+(len(MoreThanExamples)*getImpurityOfExamples(MoreThanExamples,method)))/len(examplesForCurAttr)
    
class Node(object):
    def __init__(self,examples,decisionAttributeType,decisionAttribute,decisionAttributeValue,level):
        self.examples = examples
        self.decisionAttributeType = decisionAttributeType
        self.decisionAttribute = decisionAttribute        
        self.decisionAttributeValue = decisionAttributeValue
        self.level = level
        self.leftChild = None
        self.rightChild = None
        
    def __str__(self): 
        return "Decision attribute type is "+self.decisionAttributeType+"decisionAttribute is "+ self.decisionAttribute+" decisionAttributeValue is "+ str(self.decisionAttributeValue)+" of level "+str(self.level)+" number of examples of this node "+str(len(self.examples))
    
def BestOfAttributes(examplesForCurAttr,targetAttribute,method):
    
    fra1 = train.loc[examplesForCurAttr,CategoricalAttributes]
    dictForCategAttrUniqueValues = {}
    for i in CategoricalAttributes:
        dictForCategAttrUniqueValues[i] = list(fra1[i].sort_values().unique())
    dictForEntropyUniqueValueCateg = copy.deepcopy(dictForCategAttrUniqueValues)
    print("calculating Impurities please wait")
    for i in CategoricalAttributes:
        for j,val in enumerate(dictForCategAttrUniqueValues[i]):
            dictForEntropyUniqueValueCateg[i][j] = getMethodImpurity(i,dictForCategAttrUniqueValues[i][j],examplesForCurAttr,method)          
    EntropyValuesForCategoricalAttributes = {}
    for i in CategoricalAttributes:
        EntropyValuesForCategoricalAttributes[i]=min(dictForEntropyUniqueValueCateg[i])
    BestOfCategoricalAttributes = min(EntropyValuesForCategoricalAttributes, key=EntropyValuesForCategoricalAttributes.get)
    BestOfCategoricalAttributesValue = min(EntropyValuesForCategoricalAttributes.values())   
    
    if BestOfCategoricalAttributesValue > 1:
        return BestOfCategoricalAttributes,'No Attribute'
        
    for index, item in enumerate(dictForEntropyUniqueValueCateg[BestOfCategoricalAttributes]):
        if item == BestOfCategoricalAttributesValue:
            BestSplitValueCategorical = dictForCategAttrUniqueValues[BestOfCategoricalAttributes][index]
            break            
    return BestOfCategoricalAttributes,BestSplitValueCategorical
    
def DecisionTreeBuilder(examplesForCurNode,targetAttribute,level,method):
    allExPos = True
    for i in examplesForCurNode:
        if str(train.at[int(i),targetAttribute]) == str(1):
            pass
        else:
            allExPos = False
            break
    if allExPos:        
        r = Node(examplesForCurNode,'leafNode','leafNode','Positive',level)
        print(r)
        #print("Type1")
        return r
    allExNeg = True
    for j in examplesForCurNode:
        if str(train.at[j,targetAttribute]) == str(0):
            pass
        else:
            allExNeg = False
            break
    if allExNeg:
        r = Node(examplesForCurNode,'leafNode','leafNode','Negative',level)
        print(r)
        #print("Type2")
        return r
    if getImpurityOfExamples(examplesForCurNode,method) < 0.001 or len(examplesForCurNode)<10:
        fra1 = train.loc[examplesForCurNode,[targetAttribute] ]
        targetAttributesList = fra1[targetAttribute].values
        unique1, counts1 = np.unique(targetAttributesList, return_counts=True)
        dictOfTaAttr = dict(zip(unique1, counts1))
        posCount = dictOfTaAttr[1]
        negCount = dictOfTaAttr[0]
        if (posCount/(posCount+negCount))>0.24:
            r = Node(examplesForCurNode,'leafNode','leafNode','Positive',level)
            #numberOfNodes  = numberOfNodes + 1
        else:
            r = Node(examplesForCurNode,'leafNode','leafNode','Negative',level)
            #numberOfNodes  = numberOfNodes + 1
        print(r)
        #print("Type3")
        return r
    
    BestAttribute,BestAttributeValue = BestOfAttributes(examplesForCurNode,targetAttribute,method)
    
    if BestAttributeValue == 'No Attribute':
        fra1 = train.loc[examplesForCurNode,[targetAttribute] ]
        targetAttributesList = fra1[targetAttribute].values
        unique1, counts1 = np.unique(targetAttributesList, return_counts=True)
        dictOfTaAttr = dict(zip(unique1, counts1))
        posCount = dictOfTaAttr[1]
        negCount = dictOfTaAttr[0]
        if (posCount/(posCount+negCount))>0.24:
            r = Node(examplesForCurNode,'leafNode','leafNode','Positive',level)
            #numberOfNodes  = numberOfNodes + 1
        else:
            r = Node(examplesForCurNode,'leafNode','leafNode','Negative',level)
            #numberOfNodes  = numberOfNodes + 1
        print(r)
        print("Type8")
        return r 
    
    if BestAttribute in NumericalAttributes:
        BestAttributeType = 'numerical'
    else:
        BestAttributeType = 'categorical'
    r = Node(examplesForCurNode,BestAttributeType,BestAttribute,BestAttributeValue,level)
    print(r)
    print("Type4")
    #numberOfNodes  = numberOfNodes + 1
    examplesForLeftChildOfCurrentNode = []
    examplesForRightChildOfCurrentNode = []        
    if BestAttributeType == 'categorical':        
        for iter1 in examplesForCurNode:
            if  str(train.at[int(iter1),str(BestAttribute)]) == str(BestAttributeValue):
                examplesForLeftChildOfCurrentNode.append(int(iter1))
            else:
                examplesForRightChildOfCurrentNode.append(int(iter1))
    else:
        for iter1 in examplesForCurNode:
            if  float(str(train.at[int(iter1),str(BestAttribute)])) <= float(str(BestAttributeValue)):
                examplesForLeftChildOfCurrentNode.append(int(iter1))
            else:
                examplesForRightChildOfCurrentNode.append(int(iter1))
    fra1 = train.loc[examplesForCurNode,[targetAttribute] ]
    targetAttributesList = fra1[targetAttribute].values
    unique1, counts1 = np.unique(targetAttributesList, return_counts=True)
    dictOfTaAttr = dict(zip(unique1, counts1))
    posCount = dictOfTaAttr[1]
    negCount = dictOfTaAttr[0]
    if len(examplesForLeftChildOfCurrentNode)!=0:
        r.leftChild = DecisionTreeBuilder(examplesForLeftChildOfCurrentNode,targetAttribute,level+1,method)
    else:
        if (posCount/(posCount+negCount))>0.24:
            r.leftChild = Node(examplesForCurNode,'leafNode','leafNode','Positive',level)
        else:
            r.leftChild = Node(examplesForCurNode,'leafNode','leafNode','Negative',level)
    
    if len(examplesForRightChildOfCurrentNode)!=0:
        r.rightChild = DecisionTreeBuilder(examplesForRightChildOfCurrentNode,targetAttribute,level+1,method)
    else:
        if (posCount/(posCount+negCount))>0.24:
            r.rightChild = Node(examplesForCurNode,'leafNode','leafNode','Positive',level)
        else:
            r.rightChild = Node(examplesForCurNode,'leafNode','leafNode','Negative',level)
            
   
    return r 


In [105]:
examplesForRootNode = examples[:]
method = 'Entropy'
RootNode = DecisionTreeBuilder(examplesForRootNode,targetAttribute,0,method)

calculating Impurities please wait
{'Work_accident': 0.7745142025788776, 'promotion_last_5years': 0.7902178308710968, 'sales': 0.7919786305095348, 'salary': 0.7817877061008773}
Decision attribute type is categoricaldecisionAttribute is Work_accident decisionAttributeValue is 0 of level 0 number of examples of this node 8989
Type4
calculating Impurities please wait
{'Work_accident': 999, 'promotion_last_5years': 0.8318588433493592, 'sales': 0.8334925478478579, 'salary': 0.8212911382557005}
Decision attribute type is categoricaldecisionAttribute is salary decisionAttributeValue is low of level 1 number of examples of this node 7665
Type4
calculating Impurities please wait
{'Work_accident': 999, 'promotion_last_5years': 0.914718427109622, 'sales': 0.9117685665087817, 'salary': 999}
Decision attribute type is categoricaldecisionAttribute is sales decisionAttributeValue is RandD of level 2 number of examples of this node 3745
Type4
calculating Impurities please wait
{'Work_accident': 999, '

{'Work_accident': 999, 'promotion_last_5years': 999, 'sales': 0.8908361256499749, 'salary': 999}
Decision attribute type is categoricaldecisionAttribute is sales decisionAttributeValue is IT of level 11 number of examples of this node 513
Type4
calculating Impurities please wait
{'Work_accident': 999, 'promotion_last_5years': 999, 'sales': 999, 'salary': 999}
Decision attribute type is leafNodedecisionAttribute is leafNode decisionAttributeValue is Positive of level 12 number of examples of this node 329
Type8
calculating Impurities please wait
{'Work_accident': 999, 'promotion_last_5years': 999, 'sales': 999, 'salary': 999}
Decision attribute type is leafNodedecisionAttribute is leafNode decisionAttributeValue is Positive of level 12 number of examples of this node 184
Type8
Decision attribute type is leafNodedecisionAttribute is leafNode decisionAttributeValue is Positive of level 11 number of examples of this node 2
Type3
calculating Impurities please wait
{'Work_accident': 999, 'pr

{'Work_accident': 999, 'promotion_last_5years': 999, 'sales': 0.774523234022284, 'salary': 999}
Decision attribute type is categoricaldecisionAttribute is sales decisionAttributeValue is hr of level 4 number of examples of this node 3227
Type4
calculating Impurities please wait
{'Work_accident': 999, 'promotion_last_5years': 999, 'sales': 999, 'salary': 999}
Decision attribute type is leafNodedecisionAttribute is leafNode decisionAttributeValue is Positive of level 5 number of examples of this node 180
Type8
calculating Impurities please wait
{'Work_accident': 999, 'promotion_last_5years': 999, 'sales': 0.7604004300715839, 'salary': 999}
Decision attribute type is categoricaldecisionAttribute is sales decisionAttributeValue is technical of level 5 number of examples of this node 3047
Type4
calculating Impurities please wait
{'Work_accident': 999, 'promotion_last_5years': 999, 'sales': 999, 'salary': 999}
Decision attribute type is leafNodedecisionAttribute is leafNode decisionAttribute

{'Work_accident': 999, 'promotion_last_5years': 0.46560779209954417, 'sales': 0.46515697741489626, 'salary': 0.46580333399071205}
Decision attribute type is categoricaldecisionAttribute is sales decisionAttributeValue is product_mng of level 4 number of examples of this node 1040
Type4
calculating Impurities please wait
{'Work_accident': 999, 'promotion_last_5years': 999, 'sales': 999, 'salary': 0.3434384618746064}
Decision attribute type is categoricaldecisionAttribute is salary decisionAttributeValue is low of level 5 number of examples of this node 76
Type4
calculating Impurities please wait
{'Work_accident': 999, 'promotion_last_5years': 999, 'sales': 999, 'salary': 999}
Decision attribute type is leafNodedecisionAttribute is leafNode decisionAttributeValue is Negative of level 6 number of examples of this node 44
Type8
calculating Impurities please wait
{'Work_accident': 999, 'promotion_last_5years': 999, 'sales': 999, 'salary': 999}
Decision attribute type is leafNodedecisionAttr

{'Work_accident': 999, 'promotion_last_5years': 999, 'sales': 0.4943426417542335, 'salary': 999}
Decision attribute type is categoricaldecisionAttribute is sales decisionAttributeValue is marketing of level 11 number of examples of this node 147
Type4
calculating Impurities please wait
{'Work_accident': 999, 'promotion_last_5years': 999, 'sales': 999, 'salary': 999}
Decision attribute type is leafNodedecisionAttribute is leafNode decisionAttributeValue is Negative of level 12 number of examples of this node 36
Type8
calculating Impurities please wait
{'Work_accident': 999, 'promotion_last_5years': 999, 'sales': 0.46610492383748786, 'salary': 999}
Decision attribute type is categoricaldecisionAttribute is sales decisionAttributeValue is management of level 12 number of examples of this node 111
Type4
calculating Impurities please wait
{'Work_accident': 999, 'promotion_last_5years': 999, 'sales': 999, 'salary': 999}
Decision attribute type is leafNodedecisionAttribute is leafNode decisio

In [106]:
def RecursiveFunctionToPredict(i,r,TestSet):
    if r.decisionAttribute == 'leafNode':
        if r.decisionAttributeValue == 'Positive':
            return '1'
        else:
            return '0'
    else:
        if r.decisionAttributeType == 'categorical':            
            AttrValue = str(TestSet.at[int(i),str(r.decisionAttribute)])            
            if AttrValue == str(r.decisionAttributeValue):
                return RecursiveFunctionToPredict(i,r.leftChild,TestSet)
            else:
                return RecursiveFunctionToPredict(i,r.rightChild,TestSet)            
        elif r.decisionAttributeType == 'numerical':            
            AttrValue = float(str(TestSet.at[int(i),str(r.decisionAttribute)]))            
            if AttrValue <= float(str(r.decisionAttributeValue)):
                return RecursiveFunctionToPredict(i,r.leftChild,TestSet)
            else:
                return RecursiveFunctionToPredict(i,r.rightChild,TestSet)
def predict(RootNode,NameOfTestFile):
    GivenTestData = pd.read_csv(str(NameOfTestFile))
    GivenTestData['predict']= [0]*len(GivenTestData)
    for i in list(GivenTestData.index.values):        
        GivenTestData.at[i,'predict'] = RecursiveFunctionToPredict(i,RootNode,GivenTestData)
    return GivenTestData            

In [107]:
    GivenTestData =  train.copy()
    GivenTestData['predict']= [0]*len(GivenTestData)
    for i in list(GivenTestData.index.values):        
        GivenTestData.at[i,'predict'] = RecursiveFunctionToPredict(i,RootNode,GivenTestData)
    
    TN = 0
    FP = 0
    FN = 0
    TP = 0
    for i in list(GivenTestData.index.values):
        if str(GivenTestData.at[i,'left']) == str(1) and str(GivenTestData.at[i,'predict']) == str(1):
            TP = TP + 1
        elif str(GivenTestData.at[i,'left']) == str(1) and str(GivenTestData.at[i,'predict']) == str(0):
            FN = FN + 1
        elif str(GivenTestData.at[i,'left']) == str(0) and str(GivenTestData.at[i,'predict']) == str(1):
            FP = FP + 1
        elif str(GivenTestData.at[i,'left']) == str(0) and str(GivenTestData.at[i,'predict']) == str(0):
            TN = TN + 1
    print("Results On Training Data")
    print("Using MissClassification Rate as impurity measure")
    print("-------------------------------------------------")
    print("True Positive are    "+ str(TP))
    print("True Negatives are   "+ str(TN))
    print("False Positive are   "+ str(FP))
    print("False Negatives are  "+str(FN))
    
    Precision = (TP/(TP+FP))
    Recall = (TP/(TP+FN))
    F1_Score = 2*((1/Recall)+(1/Precision))
    accuracy = (TN+TP)/(TN+TP+FP+FN)
    print("Precsion is          "+str(Precision))
    print("Recall is            "+str(Recall))
    print("F1_Score is          "+str(F1_Score))
    print("Accuracy is          "+str(accuracy))
    print("-------------------------------------------------")
    print("( Here 1's  are positive 0's are negative)")
    len(train)

Results On Training Data
Using MissClassification Rate as impurity measure
-------------------------------------------------
True Positive are    1589
True Negatives are   3563
False Positive are   3273
False Negatives are  564
Precsion is          0.32682023858494447
Recall is            0.7380399442638179
F1_Score is          8.82945248584015
Accuracy is          0.5731449549449327
-------------------------------------------------
( Here 1's  are positive 0's are negative)


8989

In [108]:
    GivenTestData =  val.copy()
    GivenTestData['predict']= [0]*len(GivenTestData)
    for i in list(GivenTestData.index.values):        
        GivenTestData.at[i,'predict'] = RecursiveFunctionToPredict(i,RootNode,GivenTestData)
    
    TN = 0
    FP = 0
    FN = 0
    TP = 0
    for i in list(GivenTestData.index.values):
        if str(GivenTestData.at[i,'left']) == str(1) and str(GivenTestData.at[i,'predict']) == str(1):
            TP = TP + 1
        elif str(GivenTestData.at[i,'left']) == str(1) and str(GivenTestData.at[i,'predict']) == str(0):
            FN = FN + 1
        elif str(GivenTestData.at[i,'left']) == str(0) and str(GivenTestData.at[i,'predict']) == str(1):
            FP = FP + 1
        elif str(GivenTestData.at[i,'left']) == str(0) and str(GivenTestData.at[i,'predict']) == str(0):
            TN = TN + 1
    print("Results On Validation Data")
    print("Using Entropy as impurity measure")
    print("-------------------------------------------------")
    print("True Positive are    "+ str(TP))
    print("True Negatives are   "+ str(TN))
    print("False Positive are   "+ str(FP))
    print("False Negatives are  "+str(FN))
    
    Precision = (TP/(TP+FP))
    Recall = (TP/(TP+FN))
    F1_Score = 2*((1/Recall)+(1/Precision))
    accuracy = (TN+TP)/(TN+TP+FP+FN)
    print("Precsion is          "+str(Precision))
    print("Recall is            "+str(Recall))
    print("F1_Score is          "+str(F1_Score))
    print("Accuracy is          "+str(accuracy))
    print("-------------------------------------------------")
    print("( Here 1's  are positive 0's are negative)")
    len(val)

Results On Validation Data
Using Entropy as impurity measure
-------------------------------------------------
True Positive are    405
True Negatives are   888
False Positive are   839
False Negatives are  117
Precsion is          0.32556270096463025
Recall is            0.7758620689655172
F1_Score is          8.720987654320988
Accuracy is          0.5749221876389506
-------------------------------------------------
( Here 1's  are positive 0's are negative)


2249